# Boiler/Turbo-Generator System

*Optimization of Chemical Processes*, Edgar & Himmelblau (2001) - Example 11.4

## Statement

Two turbo-generators (TG) are used to supply electric power and process steam to an industrial unit. The TGs are designed such that their capacity to collect steam at each pressure level differ.

Consider each TG is fed high pressure steam and and has three possible extraction points: medium pressure steam, low pressure steam, and condensate. The electric power generated assumes a 100% thermal efficiency converting the difference between total enthalpy in the inlet and outlet of the TGs.

To meet industrial demands for medium and low pressure steam, two bypass valves might be opened. The first takes high pressure steam from source and converts it into medium pressure steam. The second converts medium pressure steam into low pressure.

| Steam Pressure | Enthalpy [MWh/ton] |
| --- | --- |
| High | 0.878 |
| Medium | 0.819 |
| Low | 0.808 |
| Condensate | 0.125 |

$ $

| Resource | Demand |
| --- | --- |
| Medium Steam [ton/h] | 123.2 |
| Low Steam [ton/h] | 45.6 |
| Electric Power [MW] | 24.55 |

$ $

| Property | TG1 | TG2 |
| --- | --- | --- |
| Max Gen [MW] | 6.25 | 9.00 |
| Min Gen [MW] | 2.50 | 3.00 |
| Max inlet [ton/h] | 87.09 | 110.67 |
| Max condensate [ton/h] | 28.12 | 0.00 |
| Max internal* [ton/h] | 59.87 | 64.41 |

$ $

| Property | Value |
| --- | --- |
| Fuel Cost [$/MWh] | 5.73 |
| Boiler Efficiency | 0.75 |
| Steam cost** [$/ton] | 5.75 |
| Electric base cost [$/MWh] | 23.90 |
| Penalty below 12MW [$/MWh] | 9.83 |

*Extracted at low pressure + condensate

**Steam cost equation:

$5.73 / 0.75 \text{\$/MWh} \cdot (0.878 - 0.125) \text{MWh/ton} = 5.75 \text{\$/ton}$

In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo

## Optimization problem

In [2]:
model = pyo.ConcreteModel()

## Sets
- $T$ : Turbogenerators
- $S$ : Steam levels

In [3]:
model.T = pyo.Set(initialize=[1, 2])
model.S = pyo.Set(initialize=["hi", "med", "low", "cond"])

## Parameters

In [4]:
# Power limits per TG
model.max_gen = pyo.Param(model.T, initialize={1: 6.25, 2: 9.0})
model.min_gen = pyo.Param(model.T, initialize={1: 2.5, 2: 3.0})

# Flow limits per TG
model.max_inlet = pyo.Param(model.T, initialize={1: 87.09, 2: 110.67})
model.max_internal = pyo.Param(model.T, initialize={1: 59.87, 2: 64.41})
model.max_condensate = pyo.Param(model.T, initialize={1: 28.12, 2: 0.0})

# Steam and power demands
model.mid_dem = pyo.Param(initialize=123.2)
model.low_dem = pyo.Param(initialize=45.6)
model.ele_dem = pyo.Param(initialize=24.55)

# Costs
model.steam_cost = pyo.Param(initialize=5.75)
model.ele_cost = pyo.Param(initialize=23.9)
model.pen_cost = pyo.Param(initialize=9.83)
model.min_contract = pyo.Param(initialize=12.0)

# Enthalpy
model.enthalpy = pyo.Param(model.S, initialize={"hi":0.878, "med":0.819, "low":0.808, "cond":0.125})

## Decision Variables

In [5]:
# Power generation must be within bounds
def rule_power_bounds(model, t):
    return (model.min_gen[t], model.max_gen[t])

# Steam flow
model.steam_cons = pyo.Var(within=pyo.NonNegativeReals)
model.flow = pyo.Var(model.T, model.S, within=pyo.NonNegativeReals)

# By-pass valves
model.bypass_med = pyo.Var(within=pyo.NonNegativeReals)
model.bypass_low = pyo.Var(within=pyo.NonNegativeReals)

# Electrical power
model.power_gen = pyo.Var(model.T, bounds=rule_power_bounds, within=pyo.NonNegativeReals)
model.purch_power = pyo.Var(within=pyo.NonNegativeReals)
model.penalty_power = pyo.Var(within=pyo.NonNegativeReals)

## Constraints

### Material Balances

In [6]:
# Balance for medium extraction
def rule_med_gen(model):
    return sum(model.flow[t, "med"] for t in model.T) + (model.bypass_med - model.bypass_low)

model.med_gen = pyo.Expression(rule=rule_med_gen)

# Balance for low extraction
def rule_low_gen(model):
    return sum(model.flow[t, "low"] for t in model.T) + model.bypass_low

model.low_gen = pyo.Expression(rule=rule_low_gen)

# Balance on the inlet
def rule_steam_balance_inlet(model):
    return model.steam_cons == sum(model.flow[t, "hi"] for t in model.T) + model.bypass_med

model.cstr_steam_balance_inlet = pyo.Constraint(rule=rule_steam_balance_inlet)

# Global balance
def rule_steam_balance_global(model):
    return model.steam_cons == sum(model.flow[t, "cond"] for t in model.T) + model.med_gen + model.low_gen

model.cstr_steam_balance_global = pyo.Constraint(rule=rule_steam_balance_global)

# Balance per turbine
def rule_steam_turbine(model, t):
    return 2 * model.flow[t, "hi"] == sum(model.flow[t, s] for s in model.S)

model.cstr_steam_turbine = pyo.Constraint(model.T, rule=rule_steam_turbine)

### Energy Balance

In [7]:
def rule_energy_balance(model, t):
    return 2 * (model.enthalpy["hi"] * model.flow[t, "hi"]) \
        - sum(model.enthalpy[s] * model.flow[t, s] for s in model.S) \
            == model.power_gen[t]
            
model.cstr_energy_balance = pyo.Constraint(model.T, rule=rule_energy_balance)

### Turbogenerator capacity constraints

In [8]:
# Max inlet
def rule_max_inlet(model, t):
    return model.flow[t, "hi"] <= model.max_inlet[t]

model.cstr_max_inlet = pyo.Constraint(model.T, rule=rule_max_inlet)

# Max internal
def rule_max_internal(model, t):
    return model.flow[t, "cond"] + model.flow[t, "low"] <= model.max_internal[t]

model.cstr_max_internal = pyo.Constraint(model.T, rule=rule_max_internal)

# Max condensate
def rule_max_cond(model, t):
    return model.flow[t, "cond"] <= model.max_condensate[t]

model.cstr_max_cond = pyo.Constraint(model.T, rule=rule_max_cond)

### Energy demands

In [9]:
# Min med gen
def rule_mid_dem(model):
    return model.mid_dem <= model.med_gen

model.cstr_mid_dem = pyo.Constraint(rule=rule_mid_dem)

# Min low gen
def rule_low_dem(model):
    return model.low_dem <= model.low_gen

model.cstr_low_dem = pyo.Constraint(rule=rule_low_dem)

# Power demand
def rule_ele_dem(model):
    return model.ele_dem <= sum(model.power_gen[t] for t in model.T) + model.purch_power

model.cstr_ele_dem = pyo.Constraint(rule=rule_ele_dem)

# Purchase power contract
def rule_ele_contract(model):
    return model.min_contract <= model.penalty_power + model.purch_power

model.cstr_ele_contract = pyo.Constraint(rule=rule_ele_contract)

## Objective

In [10]:
# Minimize operational costs
def obj_fun(model):
    return model.steam_cost * model.steam_cons \
        + model.ele_cost * model.purch_power\
            + model.pen_cost * model.penalty_power

model.obj = pyo.Objective(rule=obj_fun, sense=pyo.minimize)

In [11]:
# model.write("turbogenerators.lp", io_options={'symbolic_solver_labels': True})

## Solve

In [12]:
cbc = pyo.SolverFactory("cbc")

In [13]:
sol = cbc.solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  5 2022 

command line - C:\Users\Bruno\Documents\Programas\Cbc\bin\cbc.exe -printingOptions all -import C:\Users\Bruno\AppData\Local\Temp\tmpgvidye8z.pyomo.lp -stat=1 -solve -solu C:\Users\Bruno\AppData\Local\Temp\tmpgvidye8z.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 10 (-7) rows, 13 (-3) columns and 37 (-14) elements
Statistics for presolved model


Problem has 10 rows, 13 columns (5 with objective) and 37 elements
There are 2 singletons with objective 
Column breakdown:
7 of type 0.0->inf, 4 of type 0.0->up, 0 of type lo->inf, 
2 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
5 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
1 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
3 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
1 of type L other, 0 of type Range 0.0->1.0, 0 of type R

In [14]:
model.obj.display()

obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 1268.6493959


In [15]:
model.flow.display()

flow : Size=8, Index=flow_index
    Key         : Lower : Value     : Upper : Fixed : Stale : Domain
    (1, 'cond') :     0 : 3.7454582 :  None : False : False : NonNegativeReals
      (1, 'hi') :     0 : 61.875458 :  None : False : False : NonNegativeReals
     (1, 'low') :     0 :       0.0 :  None : False : False : NonNegativeReals
     (1, 'med') :     0 :     58.13 :  None : False : False : NonNegativeReals
    (2, 'cond') :     0 :       0.0 :  None : False : False : NonNegativeReals
      (2, 'hi') :     0 :    110.67 :  None : False : False : NonNegativeReals
     (2, 'low') :     0 :      45.6 :  None : False : False : NonNegativeReals
     (2, 'med') :     0 :     65.07 :  None : False : False : NonNegativeReals


In [16]:
model.power_gen.display()

power_gen : Size=2, Index=T
    Key : Lower : Value   : Upper : Fixed : Stale : Domain
      1 :   2.5 :    6.25 :  6.25 : False : False : NonNegativeReals
      2 :   3.0 : 7.03113 :   9.0 : False : False : NonNegativeReals


In [17]:
model.med_gen(model)

123.19999999999999